### In this notebook I want to see how different annealing paths affect the final probability of success for a system of qubits with a natural divide

We are using the same structure as reported in Nick's Circuit Design paper. 4 ancilla bits and 4 logical bits used to construct an eefecting 4 body coupling. We will apply 2 different schedules to the transverse field so that we have $A_{ancilla}(s)$ and $A_{logical}(s)$ but they will share the same $B(s)$.

In [ ]:
import project_lib as mylib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.image as mpimg
%matplotlib inline
img1=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\circuits_paper_connection_graph.png")
plt.figure(figsize=(6,6)); plt.imshow(img1)
plt.axis('off')
plt.show()

The condition that must be met to construct the effective 4 body hamiltonian are: 

\begin{equation}
    J= J_a.
\end{equation}

\begin{equation}
    h = -J_a+q_0.
\end{equation}

\begin{equation}
    h_{a,i} = -J_a(2i-N)+q_i
\end{equation}
    where
\begin{equation}
    q_i=
    \begin{cases}
      +J_N+q_0, & \text{ if } N-i \text{ is odd }  \\
      -J_N+q_0, & \text{ if } N-i \text{ is even }
    \end{cases}
\end{equation}

\begin{equation}
0<q_0<J_a.
\end{equation}

\begin{equation}
\mid J_N \mid<<q_0<J_a.|
\end{equation}

In [ ]:
#define number of logical and ancilla qubits
log_qub = 4
anc_qub = log_qub
qubits = log_qub*2

# define the coupling parameters that comply with the conditions above
JN = 0.1
q0 = 50.
Ja = 100.
Jl = Ja
hl = -Ja+q0


# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical qubits
ha = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) # h for the ancilla qubits
h = np.concatenate((hl,ha)) # the full h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J = np.concatenate((Jt,Jb),axis=0) # the full J

In [ ]:
import matplotlib.image as mpimg
%matplotlib inline
img1=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\Jup010 - Anneal_path_graph.png")
plt.figure(figsize=(6,6)); plt.imshow(img1)
plt.axis('off')
plt.show()

I want to carry out a bunch of anneals where the annealing schedule A path with have the above form. I want them to be simply a linear schedule where they are related by
\begin{equation}
    A_{anc}(s)=(1-s) \sin(\theta)
\end{equation}

\begin{equation}
    A_{log}(s)=(1-s) \cos(\theta)
\end{equation}

\begin{equation}
    \text{and so }\\
    \sqrt{A_{anc}^2(s)+A_{log}^2(s)}=(1-s)
\end{equation}

In [ ]:
# lets start by running a straight forward anneal to see if we are going to be in the right ballpark here and also..
# .. to think about how long this will take to run. This will be the equivalent of theta = pi/4
a0 = mylib.Anneal(qubits,[h,J],T=100,points = 1000)
a0.run()
a0.show_results()

In [ ]:
#now lets run it with the intended annealing schedules. This is the neutral hamiltonian for 4 logical qubits
thetas_priority = np.asarray([1,50,100,25,75,10,40,60,90])
thetas_all = np.linspace(1,100,100)
mask = np.in1d(thetas_all, thetas_priority)
thetas_unordered = np.concatenate((thetas_priority,thetas_all[np.logical_not(mask)]))*np.pi/200
an = {}
last_prob = []
for x in thetas_unordered:
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter010.001')

plotted in the file Jupyter011

In [ ]:
# I should have used this instead. The index 100 isn't a good choice since cos(theta)=0 for this
thetas_priority = np.asarray([1,50,99,25,75,10,40,60,90])
thetas_all = np.linspace(1,99,99)
mask = np.in1d(thetas_all, thetas_priority)
thetas_unordered = np.concatenate((thetas_priority,thetas_all[np.logical_not(mask)]))
print thetas_unordered

We will try this whole thing again but with the non-neutral hamiltonian

In [ ]:
# we are going to run the same thing again but with the non-neutral hamiltonian

#define number of logical and ancilla qubits
log_qub = 4
anc_qub = log_qub
qubits = log_qub*2

# define the coupling parameters that comply with the conditions above
JN = 0.1
q0 = 50.
Ja = 100.
Jl = Ja
hl = -Ja+q0

# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical qubits
ha = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) + np.asarray([JN if (log_qub-i)%2 == 1 else -JN for i in range(1,log_qub+1)])# h for the ancilla qubits
h = np.concatenate((hl,ha)) # the full h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J = np.concatenate((Jt,Jb),axis=0) # the full J

#now lets run it with the intended annealing schedules
thetas_priority = np.asarray([1,50,99,25,75,10,40,60,90])
thetas_all = np.linspace(1,99,99)
mask = np.in1d(thetas_all, thetas_priority)
thetas_unordered = np.concatenate((thetas_priority,thetas_all[np.logical_not(mask)]))*np.pi/200
an = {}
last_prob = []
for x in thetas_unordered:
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter010.002')

### Need to run everything below here

In [ ]:
# next I want to do a run with larger Ja so that I can see the effect of this compared to my base runs above

# we are going to everything again doing both the neutral and non-neutral hamiltonians

#define number of logical and ancilla qubits
log_qub = 4
anc_qub = log_qub
qubits = log_qub*2

# define the coupling parameters that comply with the conditions above
JN = 0.1
q0 = 50.
Ja = 1000.
Jl = Ja
hl = -Ja+q0

# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical qubits
ha_neut = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)])
ha_non_neut = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) + np.asarray([JN if (log_qub-i)%2 == 1 else -JN for i in range(1,log_qub+1)])# h for the ancilla qubits
h_neut = np.concatenate((hl,ha_neut)) # the full neutral h
h_non_neut = np.concatenate((hl,ha_non_neut)) # the full non-neutral h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J = np.concatenate((Jt,Jb),axis=0) # the full J

#now lets run it with the intended annealing schedules
thetas_priority = np.asarray([1,50,99,25,75,10,40,60,90])
thetas_all = np.linspace(1,99,99)
mask = np.in1d(thetas_all, thetas_priority)
thetas_unordered = np.concatenate((thetas_priority,thetas_all[np.logical_not(mask)]))*np.pi/200

an = {}
last_prob = []
for x in thetas_unordered:
    print 'doing run %s out of %s'%(int(np.where(thetas_unordered ==x)[0])+1, len(thetas_unordered))
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h_neut,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter010.003')
    
an = {}
last_prob = []
for x in thetas_unordered:
    print 'doing run %s out of %s'%(int(np.where(thetas_unordered ==x)[0])+1, len(thetas_unordered))
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h_non_neut,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter010.004')

In [ ]:
# next I want to do a run with even larger Ja so that I can see the effect of this compared to my base runs at the top

# we are going to everything again doing both the neutral and non-neutral hamiltonians

#define number of logical and ancilla qubits
log_qub = 4
anc_qub = log_qub
qubits = log_qub*2

# define the coupling parameters that comply with the conditions above
JN = 0.1
q0 = 50.
Ja = 10000.
Jl = Ja
hl = -Ja+q0

# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical qubits
ha_neut = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)])
ha_non_neut = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) + np.asarray([JN if (log_qub-i)%2 == 1 else -JN for i in range(1,log_qub+1)])# h for the ancilla qubits
h_neut = np.concatenate((hl,ha_neut)) # the full neutral h
h_non_neut = np.concatenate((hl,ha_non_neut)) # the full non-neutral h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J = np.concatenate((Jt,Jb),axis=0) # the full J

#now lets run it with the intended annealing schedules
thetas_priority = np.asarray([1,50,99,25,75,10,40,60,90])
thetas_all = np.linspace(1,99,99)
mask = np.in1d(thetas_all, thetas_priority)
thetas_unordered = np.concatenate((thetas_priority,thetas_all[np.logical_not(mask)]))*np.pi/200

an = {}
last_prob = []
for x in thetas_unordered:
    print 'doing run %s out of %s'%(int(np.where(thetas_unordered ==x)[0])+1, len(thetas_unordered))
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h_neut,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter010.005')
    
an = {}
last_prob = []
for x in thetas_unordered:
    print 'doing run %s out of %s'%(int(np.where(thetas_unordered ==x)[0])+1, len(thetas_unordered))
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h_non_neut,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter010.006')

In [ ]:
# next I want to do a run with more qubits. I'll increase to 5 logical and 5 ancilla

# we are going to everything again doing both the neutral and non-neutral hamiltonians

#define number of logical and ancilla qubits
log_qub = 5
anc_qub = log_qub
qubits = log_qub*2

# define the coupling parameters that comply with the conditions above
JN = 0.1
q0 = 50.
Ja = 100.
Jl = Ja
hl = -Ja+q0

# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical qubits
ha_neut = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)])
ha_non_neut = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) + np.asarray([JN if (log_qub-i)%2 == 1 else -JN for i in range(1,log_qub+1)])# h for the ancilla qubits
h_neut = np.concatenate((hl,ha_neut)) # the full neutral h
h_non_neut = np.concatenate((hl,ha_non_neut)) # the full non-neutral h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J = np.concatenate((Jt,Jb),axis=0) # the full J

#now lets run it with the intended annealing schedules
thetas_priority = np.asarray([1,50,99,25,75,10,40,60,90])
thetas_all = np.linspace(1,99,99)
mask = np.in1d(thetas_all, thetas_priority)
thetas_unordered = np.concatenate((thetas_priority,thetas_all[np.logical_not(mask)]))*np.pi/200

an = {}
last_prob = []
for x in thetas_unordered:
    print 'doing run %s out of %s'%(int(np.where(thetas_unordered ==x)[0])+1, len(thetas_unordered))
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h_neut,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter010.007')
    
an = {}
last_prob = []
for x in thetas_unordered:
    print 'doing run %s out of %s'%(int(np.where(thetas_unordered ==x)[0])+1, len(thetas_unordered))
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h_non_neut,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter010.008')

In [ ]:
# next I want to do a run with more qubits. I'll increase to 6 logical and 6 ancilla

# we are going to everything again doing both the neutral and non-neutral hamiltonians

#define number of logical and ancilla qubits
log_qub = 6
anc_qub = log_qub
qubits = log_qub*2

# define the coupling parameters that comply with the conditions above
JN = 0.1
q0 = 50.
Ja = 100.
Jl = Ja
hl = -Ja+q0

# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical qubits
ha_neut = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)])
ha_non_neut = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) + np.asarray([JN if (log_qub-i)%2 == 1 else -JN for i in range(1,log_qub+1)])# h for the ancilla qubits
h_neut = np.concatenate((hl,ha_neut)) # the full neutral h
h_non_neut = np.concatenate((hl,ha_non_neut)) # the full non-neutral h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J = np.concatenate((Jt,Jb),axis=0) # the full J

#now lets run it with the intended annealing schedules
thetas_priority = np.asarray([1,50,99,25,75,10,40,60,90])
thetas_all = np.linspace(1,99,99)
mask = np.in1d(thetas_all, thetas_priority)
thetas_unordered = np.concatenate((thetas_priority,thetas_all[np.logical_not(mask)]))*np.pi/200

an = {}
last_prob = []
for x in thetas_unordered:
    print 'doing run %s out of %s'%(int(np.where(thetas_unordered ==x)[0])+1, len(thetas_unordered))
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h_neut,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter010.009')
    
an = {}
last_prob = []
for x in thetas_unordered:
    print 'doing run %s out of %s'%(int(np.where(thetas_unordered ==x)[0])+1, len(thetas_unordered))
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h_non_neut,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter010.010')

In [ ]:
# next I want to do a run with smaller Ja and thus smaller q0 so that I can see the effect of this compared to my base runs 
# at the top

# we are going to everything again doing both the neutral and non-neutral hamiltonians

#define number of logical and ancilla qubits
log_qub = 4
anc_qub = log_qub
qubits = log_qub*2

# define the coupling parameters that comply with the conditions above
JN = 0.1
q0 = 5.
Ja = 10.
Jl = Ja
hl = -Ja+q0

# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical qubits
ha_neut = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)])
ha_non_neut = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) + np.asarray([JN if (log_qub-i)%2 == 1 else -JN for i in range(1,log_qub+1)])# h for the ancilla qubits
h_neut = np.concatenate((hl,ha_neut)) # the full neutral h
h_non_neut = np.concatenate((hl,ha_non_neut)) # the full non-neutral h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J = np.concatenate((Jt,Jb),axis=0) # the full J

#now lets run it with the intended annealing schedules
thetas_priority = np.asarray([1,50,99,25,75,10,40,60,90])
thetas_all = np.linspace(1,99,99)
mask = np.in1d(thetas_all, thetas_priority)
thetas_unordered = np.concatenate((thetas_priority,thetas_all[np.logical_not(mask)]))*np.pi/200

an = {}
last_prob = []
for x in thetas_unordered:
    print 'doing run %s out of %s'%(int(np.where(thetas_unordered ==x)[0])+1, len(thetas_unordered))
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h_neut,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter010.011')
    
an = {}
last_prob = []
for x in thetas_unordered:
    print 'doing run %s out of %s'%(int(np.where(thetas_unordered ==x)[0])+1, len(thetas_unordered))
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h_non_neut,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter010.012')

In [ ]:
# next I want to do a run with just 3 logical qubits and 3 ancilla. NOTE THAT THIS IS JUST THE REGULAR SETUP WITH 3
# LOGICAL QUBITS AND 3 ANCILLA. NOT THE MORE EFFICIENT FORM

# we are going to everything again doing both the neutral and non-neutral hamiltonians

#define number of logical and ancilla qubits
log_qub = 3
anc_qub = log_qub
qubits = log_qub*2

# define the coupling parameters that comply with the conditions above
JN = 0.1
q0 = 50.
Ja = 100.
Jl = Ja
hl = -Ja+q0

# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical qubits
ha_neut = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)])
ha_non_neut = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) + np.asarray([JN if (log_qub-i)%2 == 1 else -JN for i in range(1,log_qub+1)])# h for the ancilla qubits
h_neut = np.concatenate((hl,ha_neut)) # the full neutral h
h_non_neut = np.concatenate((hl,ha_non_neut)) # the full non-neutral h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J = np.concatenate((Jt,Jb),axis=0) # the full J

#now lets run it with the intended annealing schedules
thetas_priority = np.asarray([1,50,99,25,75,10,40,60,90])
thetas_all = np.linspace(1,99,99)
mask = np.in1d(thetas_all, thetas_priority)
thetas_unordered = np.concatenate((thetas_priority,thetas_all[np.logical_not(mask)]))*np.pi/200

an = {}
last_prob = []
for x in thetas_unordered:
    print 'doing run %s out of %s'%(int(np.where(thetas_unordered ==x)[0])+1, len(thetas_unordered))
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h_neut,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter010.013')
    
an = {}
last_prob = []
for x in thetas_unordered:
    print 'doing run %s out of %s'%(int(np.where(thetas_unordered ==x)[0])+1, len(thetas_unordered))
    sfl = lambda points: np.cos(x)*mylib.linear_schedule(points)
    sfa = lambda points: np.sin(x)*mylib.linear_schedule(points)
    sfB = mylib.linear_schedule
    sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
    a = mylib.Anneal(qubits,[h_non_neut,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)
    a.run()
    an['theta='+str(x)]=a
    last_prob.append(a.problem_x0_prob[-1])
    mylib.save_object(an,'Jupyter010.014')